In [150]:
#import useful libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import pickle
import configuration
import general_statistics as stats
import figures as figs
import matplotlib.cm as cm
from matplotlib import colors
from scipy import signal
from scipy import stats as sstats
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import scipy
cmap = cm.jet
import math
from sklearn.datasets import load_iris
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.cross_decomposition import CCA
from sklearn.model_selection import cross_val_score
import random

In [212]:
# define mouse information, and preprocessin versions from analysis.
# load data for all sessions in this mouse and compute mean activity for each neuron.

mouse = 32363       ### mouse number id
decoding_v = 1            ## decoding version, normaly equal to one
motion_correction_v = 20  ### 100 means everything was aligned, 200 means it was also equalized, 20 is for day wise
alignment_v = 3           ## alignment version, version 3 is for day wise
equalization_v = 0        ## equalization version
source_extraction_v = 1   ## source extraction version
component_evaluation_v = 1 ## component evaluation version

# here we will do analysis with no registration
# registration_v = 2        ## registration version
sf = 10                   ## sampling frequency of the original signal 
re_sf=1            ## value of resampling
session_now = 2
BINS = 70
FR_limit = 12
period = int(1 * sf / re_sf)
period_resting = period *10
fixed = 'None'
## define task for plotting. This will cahnge for other mice!!!!

if mouse == 56165 or mouse == 32364:
    if mouse == 56165:
        sessions = [1,2,4]       ## sessions for this particular mouse
    if mouse == 32364:
        sessions = [1,2]
    if session_now == 1:
        task = 'OVERLAPPING'
        colapse_behaviour = 1
        labels =['Unlabel','Rest1', 'Navigation', 'Obj1' , 'Obj2', 'Run1', 'Run2']
        colornames=['k',"r","deepskyblue","g","blue","g","blue"]
        if mouse == 32364:
            fixed = 'LR'
            object_fixed = 4
            colapse_behaviour = 1
            labels =['Unlabel','Rest1', 'Navigation', 'Overlap_object' , 'Moving_object','RunOO' , 'RunMO' ]
            colornames=['k',"r","deepskyblue","g","blue","g","blue"]   
        if mouse == 56165:
            fixed = 'UR'
            object_fixed = 5
            colapse_behaviour = 1
            labels =['Unlabel','Rest1', 'Navigation', 'Overlap_object' , 'Moving_object','RunOO' , 'RunMO' ]
            colornames=['k',"r","deepskyblue","g","blue","g","blue"]   

    else:
        if session_now == 2:
            task = 'STABLE'
            colapse_behaviour = 2
            labels =['Unlabel','Rest1', 'Navigation', 'Obj1' , 'Obj2', 'Run1', 'Run2']
            colornames=['k',"r","deepskyblue","g","blue","g","blue"]
        else:
            task = 'RANDOM'
            colapse_behaviour = 0
            labels =['Unlabel','Rest1', 'Navigation', 'Obj1' , 'Obj2', 'Run1', 'Run2']
            colornames=['k',"r","deepskyblue","g","blue","g","blue"]
            
if mouse == 32365:
    sessions = [2,3] ## sessions for this particular mouse
    if session_now == 2:
        task = 'RANDOM'
        colapse_behaviour = 0
        labels =['Unlabel','Rest1', 'Navigation', 'Obj1' , 'Obj2', 'Run1', 'Run2']
        colornames=['k',"r","deepskyblue","g","blue","g","blue"]
    else:
        if session_now == 3:
            task = 'OVERLAPPING'
            fixed = 'LR'
            object_fixed = 4
            colapse_behaviour = 1
            labels =['Unlabel','Rest1', 'Navigation', 'Overlap_object' , 'Moving_object','RunOO' , 'RunMO' ]
            colornames=['k',"r","deepskyblue","g","blue","g","blue"]           
            
if mouse == 56166:
    sessions = [1,2] ## sessions for this particular mouse
    if session_now == 1:
        task = 'RANDOM'
        colapse_behaviour = 0
        labels =['Unlabel','Rest1', 'Navigation', 'Obj1' , 'Obj2', 'Run1', 'Run2']
        colornames=['k',"r","deepskyblue","g","blue","g","blue"]

    else:
        if session_now == 2:
            task = 'OVERLAPPING'
            fixed = 'UR'
            object_fixed = 5
            colapse_behaviour = 1
            labels =['Unlabel','Rest1', 'Navigation', 'Overlap_object' , 'Moving_object','RunOO' , 'RunMO' ]
            colornames=['k',"r","deepskyblue","g","blue","g","blue"]
            
if mouse == 32366:
    sessions = [2,3] ## sessions for this particular mouse
    if session_now == 3:
        task = 'RANDOM'
        colapse_behaviour = 0
        labels =['Unlabel','Rest1', 'Navigation', 'Obj1' , 'Obj2', 'Run1', 'Run2']
        colornames=['k',"r","deepskyblue","g","blue","g","blue"]
            
if mouse == 32363:
    sessions = [1,2] ## sessions for this particular mouse
    if session_now == 1:
        task = 'RANDOM'
        colapse_behaviour = 0
        labels =['Unlabel','Rest1', 'Navigation', 'Obj1' , 'Obj2', 'Run1', 'Run2']
        colornames=['k',"r","deepskyblue","g","blue","g","blue"]
    else:
        if session_now == 2:
            task = 'OVERLAPPING'
            fixed = 'UL'
            object_fixed = 6
            colapse_behaviour = 1
            labels =['Unlabel','Rest1', 'Navigation', 'Overlap_object' , 'Moving_object','RunOO' , 'RunMO' ]
            colornames=['k',"r","deepskyblue","g","blue","g","blue"]

In [213]:
file_directory = os.environ['PROJECT_DIR'] + 'neural_analysis/data/calcium_activity_day_wise/'
timeline_file_dir = os.environ['PROJECT_DIR'] + 'neural_analysis/data/timeline/'
behaviour_dir = os.environ['PROJECT_DIR'] + 'calcium_imaging_behaviour/data/ethogram/' + f'{mouse}' + '/session_' + f'{session_now}' + '/'
behaviour_dir_unsup = os.environ['PROJECT_DIR'] + 'calcium_imaging_behaviour/data/ethogram_unsupervised/' + f'{mouse}' + '/session_' + f'{session_now}' + '/'
behaviour_dir_parameters = os.environ['PROJECT_DIR'] + 'calcium_imaging_behaviour/data/ethogram_parameters/' + f'{mouse}' + '/session_' + f'{session_now}' + '/'
tracking_dir = os.environ['PROJECT_DIR'] + 'calcium_imaging_behaviour/data/center_of_mass/' + f'{mouse}' + '/session_' + f'{session_now}' + '/'
objects_dir= os.environ['PROJECT_DIR'] + 'calcium_imaging_behaviour/data/object_positions/'
figure_path = os.environ['PROJECT_DIR'] +'neural_analysis/figures/'

In [214]:
def speed_selection(tracking = None, speed_limit = 3):
    
    x = tracking[0,:]
    y = tracking[1,:]
    vx = np.diff(x)
    vy = np.diff(y)
    speed = np.sqrt(vx*vx+vy*vy)
    index = np.where(speed > speed_limit)[0]
    
    return index

def binned_tracking(tracking = None, binnumber = 70):
    
    xpos = [int(tracking[0,i]) for i in range(tracking.shape[1])]
    ypos = [int(tracking[1,i]) for i in range(tracking.shape[1])]
    
    
    xdenom = int(np.max(xpos)/binnumber+1)
    ydenom = int(np.max(ypos)/binnumber+1)
    
    xnew = [int(xpos[i]/xdenom) for i in range(tracking.shape[1])]
    ynew = [int(ypos[i]/xdenom) for i in range(tracking.shape[1])]
    
    return xnew,ynew

In [ ]:
confusion_matrix_data_list = []
performance_sup = np.zeros((10,4))
performance_sup_std = np.zeros((10,4))

confusion_matrix_pca_list = []
performance_pca = np.zeros((10,4))
performance_pca_std = np.zeros((10,4))

confusion_matrix_cca_list = []
performance_cca = np.zeros((10,4))
performance_cca_std = np.zeros((10,4))


confusion_matrix_data_unsup_list = []
performance_unsup = np.zeros((10,4))
performance_unsup_std = np.zeros((10,4))


confusion_matrix_pca_unsup_list = []
performance_pca_unsup = np.zeros((10,4))
performance_pca_unsup_std = np.zeros((10,4))


confusion_matrix_cca_unsup_list = []
performance_cca_unsup = np.zeros((10,4))
performance_cca_unsup_std = np.zeros((10,4))

for re_sf in [1,2,3,4,5,6,7,8,9,10]:

    # define all relevant files names and load them into a list
    session = session_now

    activity_list = []
    timeline_list = []
    behaviour_list = []
    behaviour_list_unsup = []
    parameters_list = []
    tracking_list = []
    total_time = 0
    day = 0
    confusion_matrix_data = []
    confusion_matrix_pca = []
    confusion_matrix_cca = []

    confusion_matrix_data_unsup = []
    confusion_matrix_pca_unsup = []
    confusion_matrix_cca_unsup = []
    
    print('LOADING TRIALS ACTIVITY AND CREATING LIST OF ACTIVITY, TRACKING AND BEHAVIOUR')
    for trial in [1,6,11,16]:

        beh_file_name_1 = 'mouse_' + f'{mouse}' + '_session_' + f'{session}' + '_day_' + f'{day+1}' + '_likelihood_0.75_ethogram.npy'
        ## LOAD BEHAVIOUR
        behaviour = np.load(behaviour_dir + beh_file_name_1)
        reshape_behaviour = np.reshape(behaviour[:int(int(behaviour.shape[0]/re_sf)*re_sf)],(int(behaviour.shape[0]/re_sf),re_sf))
        resample_beh1 = np.reshape(scipy.stats.mode(reshape_behaviour,axis=1)[0],reshape_behaviour.shape[0])

        beh_file_name_1 = 'mouse_' + f'{mouse}' + '_session_' + f'{session}' + '_day_' + f'{day+1}' + '_likelihood_0.75_parameters.npy'
        ## LOAD PARAMETRS FOR BEHAVIOUR CLASSIFICATION
        parameters = np.load(behaviour_dir_parameters + beh_file_name_1)
        params = []
        for param in range(parameters.shape[0]):
            r1_params = np.reshape(parameters[param,:int(int(behaviour.shape[0]/re_sf)*re_sf)],(int(behaviour.shape[0]/re_sf),re_sf))
            r2_params = np.reshape(scipy.stats.mode(r1_params,axis=1)[0],reshape_behaviour.shape[0])
            params.append(r2_params)
        resample_params = np.array(params)
        
        beh_file_name_1 = 'mouse_' + f'{mouse}' + '_session_' + f'{session}' + '_day_' + f'{day+1}' + '_likelihood_0.75_unsupervised_ethogram.npy'
        ## LOAD BEHAVIOUR UNSUPERVISED
        behaviour = np.load(behaviour_dir_unsup + beh_file_name_1)
        reshape_behaviour = np.reshape(behaviour[:int(int(behaviour.shape[0]/re_sf)*re_sf)],(int(behaviour.shape[0]/re_sf),re_sf))
        resample_beh1_unsup = np.reshape(scipy.stats.mode(reshape_behaviour,axis=1)[0],reshape_behaviour.shape[0])


        ## LOAD TRACKING
        tracking_file_name_1 = 'mouse_' + f'{mouse}' + '_session_' + f'{session}' + '_day_' + f'{day+1}' + '_likelihood_0.75.npy'
        position = np.load(tracking_dir + tracking_file_name_1)
        resample_position, resample_position_stc = stats.resample_matrix(neural_activity=position.T,
                                                                                            re_sf=re_sf)
        ## LOAD TIMELINE
        time_file_session_1 =  'mouse_'+ f'{mouse}'+'_session_'+ f'{session}' +'_trial_'+ f'{trial}'+'_v1.3.1.0_10.pkl'

        timeline_file= open(timeline_file_dir + time_file_session_1,'rb')
        timeline_info = pickle.load(timeline_file)
        timeline_1 = np.zeros(len(timeline_info) + 1)
        for i in range(len(timeline_info)):
            timeline_1[i] = timeline_info[i][1]
        timeline_1[len(timeline_info)] = behaviour.shape[0]
        timeline = timeline_1/re_sf
        time_lenght = 10
        resample_timeline = timeline_1/re_sf
        timeline_list.append(resample_timeline)

        behaviour_list.append(resample_beh1)
        behaviour_list_unsup.append(resample_beh1_unsup)
        parameters_list.append(resample_params)
        tracking_list.append(resample_position)
        total_time = total_time + behaviour.shape[0]


        file_name_session_1 = 'mouse_' + f'{mouse}' + '_session_' + f'{session}' + '_trial_'+ f'{trial}'+'_v' + f'{decoding_v}' + '.4.' + f'{motion_correction_v}' + \
                              '.' + f'{alignment_v}' + '.' + f'{equalization_v}' + '.' + f'{source_extraction_v}' + '.' + \
                              f'{component_evaluation_v}' +  '.0.npy'

        ##load activity and timeline
        activity = np.load(file_directory + file_name_session_1)
        neural_activity1 = activity[1:,:]
        ## z-score neural activity
        neural_activity = neural_activity1
        ##downsample neural activity
        resample_neural_activity_mean, resample_neural_activity_std = stats.resample_matrix(neural_activity=neural_activity,
                                                                                            re_sf=re_sf)

        activity_list.append(resample_neural_activity_mean)

        day = day + 1
        
        ## define targets of behaviour according to protocol (random, overlapping and stable)

    id_target = [0,1,2,3,4,5,6] # 0: unlabeled, 1:resting, 2:Navigation, 3: Obj1, 4:Obj2, 5:Run1, 6:Run2
    
    print('REDIFINING BEHAVIOUR FOR DIFFERENT SESSIONS')

    if colapse_behaviour == 0 : # RANDOM
        for day in range(len(behaviour_list)):
            for trial in range(5):
                behaviour_trial = behaviour_list[day][int(timeline_list[day][2*trial]):int(timeline_list[day][2*trial+1])]
                objects = np.unique(behaviour_trial)
                selected_object = np.random.randint(len(objects)-4,len(objects)-2,1)
                index0 = np.where(behaviour_trial==objects[selected_object])[0]
                index1 = np.where(np.logical_and(behaviour_trial==objects[len(objects)-4], behaviour_trial!=objects[selected_object]))
                index2 = np.where(np.logical_and(behaviour_trial==objects[len(objects)-3], behaviour_trial!=objects[selected_object]))
                behaviour_trial[index0] = 3
                behaviour_trial[index1] = 4
                behaviour_trial[index2] = 4            

                index0 = np.where(behaviour_trial==objects[selected_object]+4)[0]
                index1 = np.where(np.logical_and(behaviour_trial==objects[len(objects)-2], behaviour_trial!=objects[selected_object]+4))
                index2 = np.where(np.logical_and(behaviour_trial==objects[len(objects)-1], behaviour_trial!=objects[selected_object]+4))
                behaviour_trial[index0] = 5
                behaviour_trial[index1] = 6
                behaviour_trial[index2] = 6 

                behaviour_list[day][int(timeline_list[day][2*trial]):int(timeline_list[day][2*trial+1])] = behaviour_trial


    if colapse_behaviour == 1 : #OVERLAPPING
        for day in range(len(behaviour_list)):
            behaviour_list[day][np.where(behaviour_list[day] == object_fixed)[0]] = 100
            behaviour_list[day][np.where(np.logical_and(behaviour_list[day]>=3, behaviour_list[day]<=6))[0]] = 4
            behaviour_list[day][np.where(behaviour_list[day] == 100)[0]] = 3        
            behaviour_list[day][np.where(behaviour_list[day] == object_fixed +4)[0]] = 200        
            behaviour_list[day][np.where(np.logical_and(behaviour_list[day]>=7, behaviour_list[day]<=10))[0]] = 6
            behaviour_list[day][np.where(behaviour_list[day] == 200)[0]] = 5        


    if colapse_behaviour == 2: #STABLE
        for day in range(len(behaviour_list)):
            objects = np.unique(behaviour_list[day])
            selected_object = np.random.randint(len(objects)-4,len(objects)-2,1)
            index0 = np.where(behaviour_list[day]==objects[selected_object])[0]
            index1 = np.where(np.logical_and(behaviour_list[day]==objects[len(objects)-4], behaviour_list[day]!=objects[selected_object]))
            index2 = np.where(np.logical_and(behaviour_list[day]==objects[len(objects)-3], behaviour_list[day]!=objects[selected_object]))
            behaviour_list[day][index0] = 3
            behaviour_list[day][index1] = 4
            behaviour_list[day][index2] = 4      

            index0 = np.where(behaviour_list[day]==objects[selected_object]+4)[0]
            index1 = np.where(np.logical_and(behaviour_list[day]==objects[len(objects)-2], behaviour_list[day]!=objects[selected_object]+4))
            index2 = np.where(np.logical_and(behaviour_list[day]==objects[len(objects)-1], behaviour_list[day]!=objects[selected_object]+4))
            behaviour_list[day][index0] = 5
            behaviour_list[day][index1] = 6
            behaviour_list[day][index2] = 6  


    print('DIMENSIONALITY REDUCCION AND TRAIN CLASSIFIER')
    
    for day in [0,1,2,3]:

        ### select tracking parts of the experiment
        tracking = tracking_list[day]
        index_track = np.where(behaviour_list_unsup[day]>0)[0]
        #index_rest = np.where(behaviour_list_unsup[day]==0)[0]

        ybehaviour = behaviour_list[day][index_track]
        ybehaviour_unsup = behaviour_list_unsup[day][index_track]
        yparameters = parameters_list[day][:,index_track]

        ### prepare neural data
        Xpre = activity_list[day].T
        
        #Xresting = [index_rest,:]
        Xpre = Xpre[index_track,:]
        firing_rate = np.mean(Xpre,axis = 0)*10
        selected_cells = np.where(firing_rate < FR_limit)[0]
        Xpre = Xpre[:,selected_cells]
        #X = X.astype(int)
        
        ### solve class inbalance by taking the minimun occupancy lengh supervised
        min_exploration = X.shape[0]
        for i in range(1,7):
            exploration = np.where(ybehaviour == i)[0].shape[0]
            if exploration < min_exploration and exploration!=0: min_exploration = exploration 
        
        selected_temporal_bins = []
        for i in range(1,7):
            exploration = list(np.where(ybehaviour == i)[0])
            selection = random.sample(exploration, min_exploration)
            selected_temporal_bins = selected_temporal_bins + selection
            
        X = Xpre[selected_temporal_bins,:]
        ybehaviour = ybehaviour[selected_temporal_bins]
        yparameters = yparameters[:,selected_temporal_bins]
        
        #train classifier in data
        gnb = GaussianNB()
        X_train, X_test, y_train, y_test = train_test_split(X, ybehaviour, test_size=0.1, random_state=0)
        y_pred = gnb.fit(X_train, y_train).predict(X_test)
        conf_sup = confusion_matrix(y_test,y_pred)
        scores = cross_val_score(gnb, X, ybehaviour, cv=10)
        performance_sup[re_sf-1,day] =  np.mean(scores)
        performance_sup_std[re_sf-1,day] =  np.std(scores)
                
        # do decoding but first do dim reduction with pca
        pca = PCA(n_components=13)
        X_pca = pca.fit(X).transform(X)
        X_train, X_test, y_train, y_test = train_test_split(X_pca, ybehaviour, test_size=0.1, random_state=0)
        y_pred = gnb.fit(X_train, y_train).predict(X_test)
        conf_pca = confusion_matrix(y_test,y_pred)
        scores = cross_val_score(gnb, X_pca, ybehaviour, cv=10)
        performance_pca[re_sf-1,day] =  np.mean(scores)
        performance_pca_std[re_sf-1,day] =  np.std(scores)
        
        
        # do the same decoding but firts doing a dim reduction by cca
        cca = CCA(n_components=13)
        X_cca = cca.fit(X, yparameters.T).transform(X)
        
        X_train, X_test, y_train, y_test = train_test_split(X_cca, ybehaviour, test_size=0.1, random_state=0)
        y_pred = gnb.fit(X_train, y_train).predict(X_test)
        #y_pred_proba = gnb.fit(X_train, y_train).predict_proba(X_test)
        #print("Fraction of good label points :"+ f'{ 1 - (y_test != y_pred).sum() /X_test.shape[0]}')
        #conf_sup = confusion_matrix(y_test,y_pred)
        conf_cca = confusion_matrix(y_test,y_pred)
        scores = cross_val_score(gnb, X_cca, ybehaviour, cv=10)
        performance_cca[re_sf-1,day] =  np.mean(scores)
        performance_cca_std[re_sf-1,day] =  np.std(scores)
        
        ### UNSUPERVISED
        ### solve class inbalance by taking the minimun occupancy lengh unsupervised
        min_exploration_unsup = Xpre.shape[0]
        for i in range(1,7):
            exploration = np.where(ybehaviour_unsup == i)[0].shape[0]
            if exploration < min_exploration_unsup and exploration!=0: min_exploration_unsup = exploration 
        
        selected_temporal_bins = []
        for i in range(1,7):
            exploration = list(np.where(ybehaviour_unsup == i)[0])
            selection = random.sample(exploration, min_exploration_unsup)
            selected_temporal_bins = selected_temporal_bins + selection
            
        X = Xpre[selected_temporal_bins,:]
        ybehaviour_unsup = ybehaviour_unsup[selected_temporal_bins]
        yparameters_unsup = parameters_list[day][:,selected_temporal_bins]
        
        
        ### train all data unsup
        X_train, X_test, y_train, y_test = train_test_split(X, ybehaviour_unsup, test_size=0.1, random_state=0)
        y_pred = gnb.fit(X_train, y_train).predict(X_test)
        conf_unsup = confusion_matrix(y_test,y_pred)
        performance_unsup[re_sf-1,day] =  1 - ((y_test != y_pred).sum() /X_test.shape[0])
        scores = cross_val_score(gnb, X, ybehaviour_unsup, cv=10)
        performance_unsup[re_sf-1,day] =  np.mean(scores)
        performance_unsup_std[re_sf-1,day] =  np.std(scores)
        
        ## train pca unsup
        X_pca = pca.fit(X).transform(X)        
        X_train, X_test, y_train, y_test = train_test_split(X_pca, ybehaviour_unsup, test_size=0.1, random_state=0)
        y_pred = gnb.fit(X_train, y_train).predict(X_test)
        conf_pca_unsup = confusion_matrix(y_test,y_pred)
        scores = cross_val_score(gnb, X_pca, ybehaviour_unsup, cv=10)
        performance_pca_unsup[re_sf-1,day] =  np.mean(scores)
        performance_pca_unsup_std[re_sf-1,day] =  np.std(scores)
        
        
        ## train cca unsup
        X_cca = cca.fit(X, yparameters_unsup.T).transform(X)
        X_train, X_test, y_train, y_test = train_test_split(X_cca, ybehaviour_unsup, test_size=0.1, random_state=0)
        y_pred = gnb.fit(X_train, y_train).predict(X_test)
        #y_pred_proba = gnb.fit(X_train, y_train).predict_proba(X_test)
        #print("Fraction of good label points :"+ f'{ 1 - (y_test != y_pred).sum() /X_test.shape[0]}')
        #conf_sup = confusion_matrix(y_test,y_pred)
        conf_cca_unsup = confusion_matrix(y_test,y_pred)        
        scores = cross_val_score(gnb, X_cca, ybehaviour_unsup, cv=10)
        performance_cca_unsup[re_sf-1,day] =  np.mean(scores)
        performance_cca_unsup_std[re_sf-1,day] =  np.std(scores)
        
        confusion_matrix_data.append(conf_sup)
        confusion_matrix_pca.append(conf_pca)
        confusion_matrix_cca.append(conf_cca)

        confusion_matrix_data_unsup.append(conf_unsup)
        confusion_matrix_pca_unsup.append(conf_pca_unsup)
        confusion_matrix_cca_unsup.append(conf_cca_unsup)
        
    confusion_matrix_data_list.append(confusion_matrix_data)
    confusion_matrix_pca_list.append(confusion_matrix_pca)
    confusion_matrix_cca_list.append(confusion_matrix_cca)
    
    confusion_matrix_data_unsup_list.append(confusion_matrix_data_unsup)
    confusion_matrix_pca_unsup_list.append(confusion_matrix_pca_unsup)
    confusion_matrix_cca_unsup_list.append(confusion_matrix_cca_unsup)    
    
    

LOADING TRIALS ACTIVITY AND CREATING LIST OF ACTIVITY, TRACKING AND BEHAVIOUR


In [ ]:
print(ybehaviour_unsup.shape)

In [ ]:
fig,ax = plt.subplots(3,2,figsize = (15,20))
colornames=["r","deepskyblue","g","magenta"]

for day in range(4):
    ax[0,0].plot(np.arange(1,11),performance_sup[:,day],color = colornames[day], marker='o', linestyle='-')
    ax[0,0].fill_between(np.arange(1,11),performance_sup[:,day]-performance_sup_std[:,day], performance_sup[:,day]+performance_sup_std[:,day],alpha = 0.1,edgecolor=colornames[day], facecolor=colornames[day])

    ax[1,0].plot(np.arange(1,11),performance_pca[:,day],color = colornames[day], marker='v', linestyle='--')
    ax[1,0].fill_between(np.arange(1,11),performance_pca[:,day]-performance_pca_std[:,day], performance_pca[:,day]+performance_pca_std[:,day],alpha = 0.1,edgecolor=colornames[day], facecolor=colornames[day])

    ax[2,0].plot(np.arange(1,11),performance_cca[:,day],color = colornames[day], marker='*', linestyle='-.')
    ax[2,0].fill_between(np.arange(1,11),performance_cca[:,day]-performance_cca_std[:,day], performance_cca[:,day]+performance_cca_std[:,day],alpha = 0.1,edgecolor=colornames[day], facecolor=colornames[day])

    ax[0,1].plot(np.arange(1,11),performance_unsup[:,day],color = colornames[day], marker='o', linestyle='-')
    ax[0,1].fill_between(np.arange(1,11),performance_unsup[:,day]-performance_unsup_std[:,day], performance_unsup[:,day]+performance_unsup_std[:,day],alpha = 0.1,edgecolor=colornames[day], facecolor=colornames[day])
    
    ax[1,1].plot(np.arange(1,11),performance_pca_unsup[:,day],color = colornames[day], marker='v', linestyle='--')
    ax[1,1].fill_between(np.arange(1,11),performance_pca_unsup[:,day]-performance_pca_unsup_std[:,day], performance_pca_unsup[:,day]+performance_pca_unsup_std[:,day],alpha = 0.1,edgecolor=colornames[day], facecolor=colornames[day])
    
    ax[2,1].plot(np.arange(1,11),performance_cca_unsup[:,day],color = colornames[day], marker='*', linestyle='-.')
    ax[2,1].fill_between(np.arange(1,11),performance_cca_unsup[:,day]-performance_cca_unsup_std[:,day], performance_cca_unsup[:,day]+performance_cca_unsup_std[:,day],alpha = 0.1,edgecolor=colornames[day], facecolor=colornames[day])

for i in range(3):
    for j in range(2):
        ax[i,j].set_xlabel('Resampling factor (Bin Size)', fontsize = 12)
        ax[i,j].set_ylabel('Performace', fontsize = 12)
        ax[i,j].set_ylim([0,0.7])
        ax[i,j].hlines(1/6, 0, 11, colors='k', linestyles='--') 
        ax[i,j].legend(['day1','day2','day3','day4'], fontsize = 15)
                        
ax[0,0].set_title('Supervised Ethogram ALL DATA', fontsize = 20)
ax[0,1].set_title('kmeans Ethogram ALL DATA',fontsize = 20)

ax[1,0].set_title('Supervised Ethogram DIM RED:PCA', fontsize = 20)
ax[1,1].set_title('kmeans Ethogram DIM RED:PCA',fontsize = 20)

ax[2,0].set_title('Supervised Ethogram DIM RED: CCA', fontsize = 20)
ax[2,1].set_title('kmeans Ethogram DIM RED: CCA',fontsize = 20)


figure_name = figure_path +'decorder_pca_cca_binzise'+f'{mouse}'+'_session_'+f'{session}'+'.png'

fig.savefig(figure_name)

In [ ]:
fig,ax = plt.subplots(4,10)

conditions = labels[1:]
for day in [0,1,2,3]:
    for re_sf in [1,2,3,4,5,6,7,8,9,10]:
        plot_matrix = confusion_matrix_cca_list[re_sf-1][day].copy()
        #print(plot_matrix)
        plot_matrix = plot_matrix# / np.sum(plot_matrix,axis = 1)
        
        for i in range(6):
            plot_matrix[i,i]=0
        ax[day,re_sf-1].imshow(plot_matrix, origin = 'lower', cmap = 'jet')
        
        x_pos = np.arange(0,len(conditions))
        y_pos = np.arange(0,len(conditions))
        
        ax[day,re_sf-1].set_yticks(y_pos)
        ax[day,re_sf-1].set_yticklabels([])
        if re_sf == 1:
            ax[day,re_sf-1].set_yticklabels(conditions)
            ax[day,re_sf-1].set_ylabel('Conditions', fontsize = 12)

        ax[day,re_sf-1].set_xticks(x_pos)
        ax[day,re_sf-1].set_xticklabels([])
        if day == 3:
            ax[day,re_sf-1].set_xticklabels(conditions)
            ax[day,re_sf-1].set_xlabel('Conditions', fontsize = 12)

        for i in range(len(conditions)):
            for j in range(len(conditions)):
                text = ax[day,re_sf-1].text(i, j, confusion_matrix_cca_list[re_sf-1][day][i,j],
                               ha="center", va="center", color="w")    
                
        plt.setp(ax[day,re_sf-1].get_xticklabels(), rotation=45, ha="right",
                 rotation_mode="anchor")
        plt.subplots_adjust(wspace=0.1, hspace=0.0)

fig.set_size_inches(50,25)   

figure_name = figure_path +'decorder_cca_confmatrix_binzise'+f'{mouse}'+'_session_'+f'{session}'+'.png'

fig.savefig(figure_name)

In [193]:
print(plot_matrix)

[[0.         0.         0.         0.5        0.                nan]
 [0.25       0.         0.33333333 0.         0.44444444        nan]
 [0.         0.         0.         0.         0.22222222        nan]
 [0.5        0.         0.         0.         0.11111111        nan]
 [0.         0.         0.66666667 0.5        0.                nan]
 [0.25       1.         0.         0.         0.22222222 0.        ]]
